In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import zipfile
%matplotlib inline

## Canonicalization

In [2]:
with open('county_and_state.csv') as f:
    county_and_state = pd.read_csv(f)
    
with open('county_and_population.csv') as f:
    county_and_pop = pd.read_csv(f)    

Suppose we'd like to join these two tables. Unfortunately, we can't, because the strings representing the county names don't match, as seen below.

In [3]:
county_and_state

,County,State
0,De Witt County,IL
1,Lac qui Parle County,MN
2,Lewis and Clark County,MT
3,St John the Baptist Parish,LS


In [4]:
county_and_pop

,County,Population
0,DeWitt,16798
1,Lac Qui Parle,8067
2,Lewis & Clark,55716
3,St. John the Baptist,43044


 Before we can join them, we'll do what I call **canonicalization**.

Canonicalization: A process for converting data that has more than one possible representation into a "standard", "normal", or canonical form (definition via Wikipedia).

In [5]:
def canonicalize_county(county_name):
    return (
        county_name
        .lower()               # lower case
        .replace(' ', '')      # remove spaces
        .replace('&', 'and')   # replace &
        .replace('.', '')      # remove dot
        .replace('county', '') # remove county
        .replace('parish', '') # remove parish
    )

In [6]:
county_and_pop['clean_county'] = county_and_pop['County'].map(canonicalize_county)
county_and_state['clean_county'] = county_and_state['County'].map(canonicalize_county)

In [7]:
county_and_state

,County,State,clean_county
0,De Witt County,IL,dewitt
1,Lac qui Parle County,MN,lacquiparle
2,Lewis and Clark County,MT,lewisandclark
3,St John the Baptist Parish,LS,stjohnthebaptist


In [8]:
county_and_pop

,County,Population,clean_county
0,DeWitt,16798,dewitt
1,Lac Qui Parle,8067,lacquiparle
2,Lewis & Clark,55716,lewisandclark
3,St. John the Baptist,43044,stjohnthebaptist


In [9]:
county_and_pop.merge(county_and_state,
                     left_on = 'clean_county', right_on = 'clean_county')

,County_x,Population,clean_county,County_y,State
0,DeWitt,16798,dewitt,De Witt County,IL
1,Lac Qui Parle,8067,lacquiparle,Lac qui Parle County,MN
2,Lewis & Clark,55716,lewisandclark,Lewis and Clark County,MT
3,St. John the Baptist,43044,stjohnthebaptist,St John the Baptist Parish,LS


## Processing Data from a Text Log Using Basic Python

In [10]:
with open('log.txt', 'r') as f:
    log_lines = f.readlines()

In [11]:
log_lines

['169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n',
 '193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] "GET /stat141/Notes/dim.html HTTP/1.0" 404 302 "http://eeyore.ucdavis.edu/stat141/Notes/session.html"\n',
 '169.237.46.240 - "" [3/Feb/2006:10:18:37 -0800] "GET /stat141/homework/Solutions/hw1Sol.pdf HTTP/1.1"\n']

Suppose we want to extract the day, month, year, hour, minutes, seconds, and timezone. Looking at the data, we see that these items are not in a fixed position relative to the beginning of the string. That is, slicing by some fixed offset isn't going to work.

In [12]:
log_lines[0]

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

In [13]:
log_lines[0][20:31]

'26/Jan/2014'

In [14]:
log_lines[1]

'193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] "GET /stat141/Notes/dim.html HTTP/1.0" 404 302 "http://eeyore.ucdavis.edu/stat141/Notes/session.html"\n'

In [15]:
log_lines[1][20:31]

'/Feb/2005:1'

Instead, we'll need to use some more sophisticated thinking. Let's focus on only the first line of the file.

In [16]:
first = log_lines[0]
first

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

In [17]:
first.split("[")
# two parts returned

['169.237.46.168 - - ',
 '26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n']

In [18]:
first.split("[")[1]
# just the second part returned

'26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

In [19]:
first.split("[")[1].split(']')
# this returns two parts

['26/Jan/2014:10:47:58 -0800',
 ' "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n']

In [20]:
first.split("[")[1].split(']')[0]
# this returns just the part between the two [ ]

'26/Jan/2014:10:47:58 -0800'

In [21]:
pertinent = first.split("[")[1].split(']')[0]
day, month, rest = pertinent.split('/')
year, hour, minute, rest = rest.split(':')
seconds, time_zone = rest.split(' ')
day, month, year, hour, minute, seconds, time_zone
# getting to the parts of the date and time

('26', 'Jan', '2014', '10', '47', '58', '-0800')

A much more sophisticated but common approach is to extract the information we need using a regular expression. See [today's lecture slides](https://docs.google.com/presentation/d/1omFKPsCaPf58VLo33U9ipRq4vt1Kj_3y9yHDUl8qYhk/edit#slide=id.g41d831233f_1_119) for more on regular expressions.

In [22]:
import re

In [23]:
patternSimple = r'\[(.*)\]'
output1 = re.search(patternSimple, first)
output1.groups()

('26/Jan/2014:10:47:58 -0800',)

In [24]:
pattern = r'\[(\d+)/(\w+)/(\d+):(\d+):(\d+):(\d+) (.+)\]'
output = re.search(pattern, first).groups()
output

('26', 'Jan', '2014', '10', '47', '58', '-0800')

In [25]:
patternMoon = r'm(uu(uu)*|oo(oo)*)n'

# Search for the pattern in the string
match = re.search(patternMoon, "moooon")

# If the pattern is found, print the matching text
if match:
    print(match.group())


moooon


In [26]:
#pattern2 = r'\(m(uu(uu)*|oo(oo)*)n)'
#this line was incorrect
output2 = re.search(patternMoon, "moooon")
output2.group()


'moooon'

In [27]:


day, month, year, hour, minute, second, time_zone = re.search(pattern, first).groups()
year, month, day, hour, minute, second, time_zone

('2014', 'Jan', '26', '10', '47', '58', '-0800')

Or alternately using the `findall` method:

In [28]:
import re

In [29]:
findall_output = re.findall(pattern, first)
findall_output

[('26', 'Jan', '2014', '10', '47', '58', '-0800')]

In [30]:

#pattern = r'\[(\d+)/(\w+)/(\d+):(\d+):(\d+):(\d+) (.+)\]'
day, month, year, hour, minute, second, time_zone = re.findall(pattern, first)[0]
year, month, day, hour, minute, second, time_zone

('2014', 'Jan', '26', '10', '47', '58', '-0800')

Note: We can return the results as a Series:

In [31]:
cols = ["Day", "Month", "Year", "Hour", "Minute", "Second", "Time Zone"]
def log_entry_to_series(line):
    return pd.Series(re.search(pattern, line).groups(), index = cols)

first_line_series = log_entry_to_series(first)
first_line_series

Day             26
Month          Jan
Year          2014
Hour            10
Minute          47
Second          58
Time Zone    -0800
dtype: object

And using this function we can create a DataFrame of all the time information.

In [32]:
log_info = pd.DataFrame(first_line_series)

In [33]:
print(pd.__version__)

2.0.3


In [34]:

for line in log_lines:
    s2 = log_entry_to_series(line)
    df2 = pd.DataFrame(s2)
    log_info = pd.concat([log_info, df2])

In [35]:
log_info

,0
Day,26
Month,Jan
Year,2014
Hour,10
Minute,47
Second,58
Time Zone,-0800
Day,26
Month,Jan
Year,2014


### Regular Expression From Lecture

Fill in the regex below so that after code executes, day is “26”, month is “Jan”, and year is “2014”. 

In [36]:
log_lines[0]

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

In [37]:
pattern = r"YOUR REGEX HERE"
matches = re.findall(pattern, log_lines[0])
#day, month, year = matches[0]
#day, month, year

## Real World Example #1: Restaurant Data

In this example, we will show how regexes can allow us to track quantitative data across categories defined by the appearance of various text fields.

In this example we'll see how the presence of certain keywords can affect quantitative data, e.g. how do restaurant health scores vary as a function of the number of violations that mention "vermin"?

In [38]:
vio = pd.read_csv('violations.csv', header=0, names=['id', 'date', 'desc'])
desc = vio['desc']
vio.head()

,id,date,desc
0,19,20171211,Inadequate food safety knowledge or lack of ce...
1,19,20171211,Unapproved or unmaintained equipment or utensils
2,19,20160513,Unapproved or unmaintained equipment or utensi...
3,19,20160513,Unclean or degraded floors walls or ceilings ...
4,19,20160513,Food safety certificate or food handler card n...


In [39]:
desc.values

array(['Inadequate food safety knowledge or lack of certified food safety manager',
       'Unapproved or unmaintained equipment or utensils',
       'Unapproved or unmaintained equipment or utensils  [ date violation corrected: 12/11/2017 ]',
       ...,
       'High risk vermin infestation  [ date violation corrected: 12/15/2017 ]',
       'Moderate risk food holding temperature   [ date violation corrected: 12/15/2017 ]',
       'Wiping cloths not clean or properly stored or inadequate sanitizer  [ date violation corrected: 12/15/2017 ]'],
      dtype=object)

In [40]:
type(desc)

pandas.core.series.Series

In [41]:
counts = desc.value_counts()

counts[:10]

desc
Unclean or degraded floors walls or ceilings                          999
Unapproved or unmaintained equipment or utensils                      659
Inadequately cleaned or sanitized food contact surfaces               493
Improper food storage                                                 476
Inadequate and inaccessible handwashing facilities                    467
Moderate risk food holding temperature                                452
Wiping cloths not clean or properly stored or inadequate sanitizer    418
Moderate risk vermin infestation                                      374
Unclean nonfood contact surfaces                                      369
Food safety certificate or food handler card not available            353
Name: count, dtype: int64

In [42]:
# Hmmm...
counts[50:60]

desc
Unclean or degraded floors walls or ceilings  [ date violation corrected: 11/29/2017 ]              16
Unclean or degraded floors walls or ceilings  [ date violation corrected: 9/19/2017 ]               16
Inadequate HACCP plan record keeping                                                                16
Unclean or degraded floors walls or ceilings  [ date violation corrected: 11/27/2017 ]              15
Unclean or degraded floors walls or ceilings  [ date violation corrected: 12/7/2017 ]               15
Inadequately cleaned or sanitized food contact surfaces  [ date violation corrected: 9/26/2017 ]    14
Unclean or degraded floors walls or ceilings  [ date violation corrected: 11/28/2017 ]              14
Unclean or degraded floors walls or ceilings  [ date violation corrected: 9/6/2017 ]                14
Unapproved or unmaintained equipment or utensils  [ date violation corrected: 9/19/2017 ]           14
Unapproved  living quarters in food facility                        

In [43]:
#Use regular expressions to cut out the extra info in square braces.
vio['clean_desc'] = (vio['desc']
             .str.replace(r'\s*\[.*\]$', '')
             .str.strip()
             .str.lower())
vio.head()

,id,date,desc,clean_desc
0,19,20171211,Inadequate food safety knowledge or lack of ce...,inadequate food safety knowledge or lack of ce...
1,19,20171211,Unapproved or unmaintained equipment or utensils,unapproved or unmaintained equipment or utensils
2,19,20160513,Unapproved or unmaintained equipment or utensi...,unapproved or unmaintained equipment or utensi...
3,19,20160513,Unclean or degraded floors walls or ceilings ...,unclean or degraded floors walls or ceilings ...
4,19,20160513,Food safety certificate or food handler card n...,food safety certificate or food handler card n...


In [44]:
vio['clean_desc'].value_counts().head() 

clean_desc
unclean or degraded floors walls or ceilings               999
unapproved or unmaintained equipment or utensils           659
inadequately cleaned or sanitized food contact surfaces    493
improper food storage                                      476
inadequate and inaccessible handwashing facilities         467
Name: count, dtype: int64

In [45]:
#use regular expressions to assign new features for the presence of various keywords
with_features = (vio
 .assign(is_clean     = vio['clean_desc'].str.contains('clean|sanit'))
 .assign(is_high_risk = vio['clean_desc'].str.contains('high risk'))
 .assign(is_vermin    = vio['clean_desc'].str.contains('vermin'))
 .assign(is_surface   = vio['clean_desc'].str.contains('wall|ceiling|floor|surface'))
 .assign(is_human     = vio['clean_desc'].str.contains('hand|glove|hair|nail'))
 .assign(is_permit    = vio['clean_desc'].str.contains('permit|certif'))
)
with_features.head()

,id,date,desc,clean_desc,is_clean,is_high_risk,is_vermin,is_surface,is_human,is_permit
0,19,20171211,Inadequate food safety knowledge or lack of ce...,inadequate food safety knowledge or lack of ce...,False,False,False,False,False,True
1,19,20171211,Unapproved or unmaintained equipment or utensils,unapproved or unmaintained equipment or utensils,False,False,False,False,False,False
2,19,20160513,Unapproved or unmaintained equipment or utensi...,unapproved or unmaintained equipment or utensi...,False,False,False,False,False,False
3,19,20160513,Unclean or degraded floors walls or ceilings ...,unclean or degraded floors walls or ceilings ...,True,False,False,True,False,False
4,19,20160513,Food safety certificate or food handler card n...,food safety certificate or food handler card n...,False,False,False,False,True,True


In [46]:
count_features = (with_features
 .groupby(['id', 'date'])
 .sum()
 .reset_index()
)
count_features.iloc[255:260, :]

,id,date,desc,clean_desc,is_clean,is_high_risk,is_vermin,is_surface,is_human,is_permit
255,489,20150728,Unclean or degraded floors walls or ceilings ...,unclean or degraded floors walls or ceilings ...,5,0,2,3,0,0
256,489,20150807,Unapproved or unmaintained equipment or utensi...,unapproved or unmaintained equipment or utensi...,1,0,0,1,0,0
257,489,20160308,High risk food holding temperature [ date vi...,high risk food holding temperature [ date vi...,2,2,1,0,1,0
258,489,20160721,Low risk vermin infestation [ date violation ...,low risk vermin infestation [ date violation ...,2,1,1,1,0,1
259,489,20161220,Inadequately cleaned or sanitized food contact...,inadequately cleaned or sanitized food contact...,3,0,1,2,0,0


In [47]:
count_features.query('is_vermin > 1')

,id,date,desc,clean_desc,is_clean,is_high_risk,is_vermin,is_surface,is_human,is_permit
255,489,20150728,Unclean or degraded floors walls or ceilings ...,unclean or degraded floors walls or ceilings ...,5,0,2,3,0,0
291,527,20170821,Inadequate and inaccessible handwashing facili...,inadequate and inaccessible handwashing facili...,1,1,2,1,1,1
1508,2622,20160526,Unclean or degraded floors walls or ceilings ...,unclean or degraded floors walls or ceilings ...,4,2,2,3,0,0
1573,2721,20150422,Foods not protected from contamination [ date...,foods not protected from contamination [ date...,2,1,2,1,0,0
1746,2945,20150921,Inadequate and inaccessible handwashing facili...,inadequate and inaccessible handwashing facili...,2,1,2,2,2,1
2277,3776,20151223,Low risk vermin infestation [ date violation ...,low risk vermin infestation [ date violation ...,2,0,2,1,0,0
3784,6729,20160706,Moderate risk vermin infestation [ date viola...,moderate risk vermin infestation [ date viola...,6,0,2,4,2,0
3949,7537,20150304,Inadequate and inaccessible handwashing facili...,inadequate and inaccessible handwashing facili...,1,0,2,1,1,0
4017,7661,20160616,Low risk vermin infestation [ date violation ...,low risk vermin infestation [ date violation ...,2,0,2,2,1,0
4165,9679,20150422,Inadequately cleaned or sanitized food contact...,inadequately cleaned or sanitized food contact...,2,0,2,2,0,0


In [48]:
#use a new pandas feature called "melt" that we won't describe in any detail
#the granularity of the resulting frame is a violation type in a given inspection
broken_down_by_violation_type = pd.melt(count_features, id_vars=['id', 'date'],
            var_name='feature', value_name='num_vios')
broken_down_by_violation_type.sort_values(["id", "date"]).head(13)

,id,date,feature,num_vios
0,19,20160513,desc,Unapproved or unmaintained equipment or utensi...
12262,19,20160513,clean_desc,unapproved or unmaintained equipment or utensi...
24524,19,20160513,is_clean,1
36786,19,20160513,is_high_risk,0
49048,19,20160513,is_vermin,0
61310,19,20160513,is_surface,1
73572,19,20160513,is_human,1
85834,19,20160513,is_permit,1
1,19,20171211,desc,Inadequate food safety knowledge or lack of ce...
12263,19,20171211,clean_desc,inadequate food safety knowledge or lack of ce...


In [49]:
#read in the scores
ins = pd.read_csv('inspections.csv',
                  header=0,
                  usecols=[0, 1, 2],
                  names=['id', 'score', 'date'])
ins.head()

,id,score,date
0,19,94,20160513
1,19,94,20171211
2,24,98,20171101
3,24,98,20161005
4,24,96,20160311


In [50]:
#join scores with the table broken down by violation type
violation_type_and_scores = (
    broken_down_by_violation_type
    .merge(ins, left_on=['id', 'date'], right_on=['id', 'date'])
)
violation_type_and_scores.head(12)

,id,date,feature,num_vios,score
0,19,20160513,desc,Unapproved or unmaintained equipment or utensi...,94
1,19,20160513,clean_desc,unapproved or unmaintained equipment or utensi...,94
2,19,20160513,is_clean,1,94
3,19,20160513,is_high_risk,0,94
4,19,20160513,is_vermin,0,94
5,19,20160513,is_surface,1,94
6,19,20160513,is_human,1,94
7,19,20160513,is_permit,1,94
8,19,20171211,desc,Inadequate food safety knowledge or lack of ce...,94
9,19,20171211,clean_desc,inadequate food safety knowledge or lack of ce...,94


In [ ]:
sns.catplot(x='num_vios', y='score',
               col='feature', col_wrap=2,
               kind='box',
               data=violation_type_and_scores)

Above we see, for example, that if a restaurant inspection involved 2 violation with the keyword "vermin", the average score for that inspection would be a little bit below 80.

## Text Processing Example 2: Police Data

In this example, we will apply string processing to the process of data cleaning and exploratory data analysis.

### Getting the Data

The city of Berkeley maintains an [Open Data Portal](https://data.cityofberkeley.info/) for citizens to access data about the city.  We will be examining [Call Data](https://data.cityofberkeley.info/Public-Safety/Berkeley-PD-Calls-for-Service/k2nh-s5h5).

<img src="calls_desc.png" width=800px />



In [ ]:
calls = pd.read_csv('Calls_for_Service.csv')
calls.head()

How many records did we get?

In [ ]:
len(calls)

What does an example `Block_Location` value look like?

In [ ]:
print(calls['Block_Location'].iloc[4])

### Preliminary observations on the data?

1. `EVENTDT` -- Contains the incorrect time
1. `EVENTTM` -- Contains the time in 24 hour format (What timezone?)
1. `CVDOW` -- Encodes the day of the week (see data documentation).
1. `InDbDate` -- Appears to be correctly formatted and appears pretty consistent in time.
1. **`Block_Location` -- a multi-line string that contains coordinates.**
1. `BLKADDR` -- Appears to be the address in `Block Location`.
1. `City` and `State` seem redundant given this is supposed to be the city of Berkeley dataset.

### Extracting locations

The block location contains geographic coordinates. Let's extract them.

In [ ]:
calls['Block_Location'][4]

In [ ]:
calls_lat_lon = (
    calls['Block_Location']
    .str.extract("\((\d+\.\d+)\, (-\d+\.\d+)\)")
)
calls_lat_lon.columns = ['Lat', 'Lon']
calls_lat_lon.head(10)

How many records have missing values?

In [ ]:
calls_lat_lon.isnull().sum()

Examine the missing values.

In [ ]:
calls[calls_lat_lon.isnull().any(axis=1)]['Block_Location'].head(10)

Join in the extracted values.

In [ ]:
calls['Lat'] = calls_lat_lon['Lat']
calls['Lon'] = calls_lat_lon['Lon']
calls.head()

## Examining Location information

Let's examine the geographic data (latitude and longitude).  Recall that we had some missing values.  Let's look at the behavior of these missing values according to crime type.

In [ ]:
missing_lat_lon = calls[calls[['Lat', 'Lon']].isnull().any(axis=1)]
missing_lat_lon['CVLEGEND'].value_counts().plot(kind='barh');

In [ ]:
calls['CVLEGEND'].value_counts().plot(kind='barh');

We might further normalize the analysis by the frequency to find which type of crime has the highest proportion of missing values.

In [ ]:
(missing_lat_lon['CVLEGEND'].value_counts() 
 / calls['CVLEGEND'].value_counts()
).sort_values(ascending=False).plot(kind="barh");

Now, let's make a crime map.

In [ ]:
# you may need to install the folium package for this to work
# to do this, uncomment the line below and run it.
# !pip install folium

In [ ]:
import folium
import folium.plugins

SF_COORDINATES = (37.87, -122.28)
sf_map = folium.Map(location=SF_COORDINATES, zoom_start=13)
locs = calls[['Lat', 'Lon']].astype('float').dropna().values
heatmap = folium.plugins.HeatMap(locs.tolist(), radius=10)
sf_map.add_child(heatmap)

### Questions

1. Is campus really the safest place to be?
1. Why are all the calls located on the street and at often at intersections?


In [ ]:
import folium.plugins

locations = calls[calls['CVLEGEND'] == 'ASSAULT'][['Lat', 'Lon']]

cluster = folium.plugins.MarkerCluster()
for _, r in locations.dropna().iterrows():
    cluster.add_child(
        folium.Marker([float(r["Lat"]), float(r["Lon"])]))
    
sf_map = folium.Map(location=SF_COORDINATES, zoom_start=13)
sf_map.add_child(cluster)
sf_map

## Bonus Content: Using pd.to_datetime to Extract Time Information

Date parsing using `pd.to_datetime`.

In [ ]:
pd.Series(log_lines).str.extract(r'\[(.*) -0800\]').apply(
    lambda s: pd.to_datetime(s, format='%d/%b/%Y:%H:%M:%S'))